<a href="https://colab.research.google.com/github/advaitjk/machine-learning-master-course/blob/master/Transfer_Learning_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning on the Cats-Dogs-Humans-Horses Dataset

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
folders = os.listdir('/content/gdrive/My Drive/images')
print(folders)

['cats', 'horses', 'humans', 'dogs']


In [0]:
image_data = []
labels = []

In [0]:
label_dict = {
    "cats" : 0,
    "horses" : 1,
    "humans" : 2,
    "dogs" : 3
}

In [0]:
for ix in folders:
  path = os.path.join("/content/gdrive/My Drive/images", ix)
  print(path)

/content/gdrive/My Drive/images/cats
/content/gdrive/My Drive/images/horses
/content/gdrive/My Drive/images/humans
/content/gdrive/My Drive/images/dogs


In [0]:
from keras.preprocessing import image

In [0]:
for ix in folders:
  path = os.path.join("/content/gdrive/My Drive/images", ix)
  for im in os.listdir(path):
    img = image.load_img(os.path.join(path, im), target_size= ((224, 224)))
    img_array = image.img_to_array(img)
    image_data.append(img_array)
    labels.append(label_dict[ix])

In [0]:
print(len(image_data), len(labels))

808 808


In [0]:
combined = list(zip(image_data, labels))

In [0]:
import random
random.shuffle(combined)

In [0]:
image_data[:], labels[:] = zip(*combined)

In [0]:
print(labels[:5])

[1, 0, 0, 2, 3]


In [0]:
import numpy as np
X_train = np.array(image_data)
Y_train = np.array(labels)
print(X_train.shape, Y_train.shape)

(808, 224, 224, 3) (808,)


In [0]:
from keras.utils import np_utils
Y_train = np_utils.to_categorical(Y_train)
print(X_train.shape, Y_train.shape)

(808, 224, 224, 3) (808, 4)


In [0]:
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.layers import *
from keras.models import Model
import matplotlib.pyplot as plt


In [0]:
model = ResNet50(include_top=False, weights = 'imagenet', input_shape = (224, 224, 3))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step


In [0]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [0]:
gap = GlobalAveragePooling2D()(model.output)
fc1 = Dense(256, activation='relu')(gap)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(4, activation='softmax')(d1)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model_new = Model(input = model.input, outputs = fc2)
model_new.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  """Entry point for launching an IPython kernel.


In [0]:
from keras.optimizers import Adam
Adam = Adam(lr=0.00003)

In [0]:
model_new.compile(loss='categorical_crossentropy', optimizer=Adam, metrics = ['accuracy'])

In [0]:
for i in range(len(model_new.layers)):
  print(i, model_new.layers[i])

0 <keras.engine.input_layer.InputLayer object at 0x7f9b9d938e48>
1 <keras.layers.convolutional.ZeroPadding2D object at 0x7f9b9d938d68>
2 <keras.layers.convolutional.Conv2D object at 0x7f9b9d9380b8>
3 <keras.layers.normalization.BatchNormalization object at 0x7f9babdf3dd8>
4 <keras.layers.core.Activation object at 0x7f9babdf32b0>
5 <keras.layers.convolutional.ZeroPadding2D object at 0x7f9b9eb87400>
6 <keras.layers.pooling.MaxPooling2D object at 0x7f9b97547c50>
7 <keras.layers.convolutional.Conv2D object at 0x7f9b96cecfd0>
8 <keras.layers.normalization.BatchNormalization object at 0x7f9b96cf40b8>
9 <keras.layers.core.Activation object at 0x7f9b96cf4da0>
10 <keras.layers.convolutional.Conv2D object at 0x7f9b96cfe6a0>
11 <keras.layers.normalization.BatchNormalization object at 0x7f9b96cb74a8>
12 <keras.layers.core.Activation object at 0x7f9b96cbfd30>
13 <keras.layers.convolutional.Conv2D object at 0x7f9b96cc7390>
14 <keras.layers.convolutional.Conv2D object at 0x7f9b96c86c88>
15 <keras.lay

In [0]:
for ix in range(169):
  model_new.layers[ix].trainable = False

In [0]:
model_new.compile(loss='categorical_crossentropy', optimizer=Adam, metrics = ['accuracy'])

In [0]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [0]:
history = model_new.fit(X_train, Y_train, shuffle = True, epochs = 5, validation_split = 0.2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 646 samples, validate on 162 samples
Epoch 1/5
646/646 [==============================] - 198s 307ms/step - loss: 1.5428 - acc: 0.3111 - val_loss: 0.7869 - val_acc: 0.7469
Epoch 2/5
646/646 [==============================] - 199s 308ms/step - loss: 0.9349 - acc: 0.6269 - val_loss: 0.4255 - val_acc: 0.9012
Epoch 3/5
646/646 [==============================] - 199s 307ms/step - loss: 0.6306 - acc: 0.7771 - val_loss: 0.2769 - val_acc: 0.9383
Epoch 4/5
646/646 [==============================] - 200s 309ms/step - loss: 0.4960 - acc: 0.8344 - val_loss: 0.2070 - val_acc: 0.9444
Epoch 5/5
646/646 [==============================] - 199s 308ms/step - loss: 0.3725 - acc: 0.8916 - val_loss: 0.1590 - val_acc: 0.9568


In [0]:
history = model_new.fit(X_train, Y_train, shuffle = True, epochs = 5, validation_split = 0.2)

Train on 646 samples, validate on 162 samples
Epoch 1/5
646/646 [==============================] - 199s 308ms/step - loss: 0.2766 - acc: 0.9365 - val_loss: 0.1429 - val_acc: 0.9506
Epoch 2/5
646/646 [==============================] - 199s 308ms/step - loss: 0.2346 - acc: 0.9489 - val_loss: 0.1275 - val_acc: 0.9506
Epoch 3/5
646/646 [==============================] - 198s 307ms/step - loss: 0.2174 - acc: 0.9334 - val_loss: 0.1013 - val_acc: 0.9877
Epoch 4/5
646/646 [==============================] - 198s 306ms/step - loss: 0.2078 - acc: 0.9551 - val_loss: 0.0912 - val_acc: 0.9815
Epoch 5/5
646/646 [==============================] - 199s 308ms/step - loss: 0.1685 - acc: 0.9520 - val_loss: 0.1019 - val_acc: 0.9568
